In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [20]:
data = pd.read_csv("ne1.csv")


In [21]:
data[1:5]

,meanfreq,sd,median,mode,Q25,Q75,IQR,skew,kurt,label
1,0.066009,0.067310,0.040229,0.000000,0.019414,0.092666,0.073252,22.423285,634.613855,male
2,0.077316,0.083829,0.036718,0.000000,0.008701,0.131908,0.123207,30.757155,1024.927705,male
3,0.151228,0.072111,0.158011,0.083878,0.096582,0.207955,0.111374,1.232831,4.177296,male
4,0.135120,0.079146,0.124656,0.104261,0.078720,0.206045,0.127325,1.101174,4.333713,male


In [22]:
classmapping={}

for idx, label in enumerate(np.unique(data['label'])):
    classmapping[label]=idx


In [23]:
classmapping

{'female': 0, 'male': 1}

In [24]:
data['label'] = data['label'].map(classmapping)

In [25]:
x,y=data.iloc[:, :-1].values,data.iloc[:, -1].values

x_train, x_test, y_train, y_test =train_test_split(x, y, test_size=0.2)

In [26]:
print(x_train.shape)
print(x_test.shape)


(2534, 9)
(634, 9)


In [27]:
x_train

array([[ 0.19752336,  0.04566586,  0.20390244, ...,  0.01931707,
         3.78202569, 18.26013103],
       [ 0.22237339,  0.05972391,  0.24193138, ...,  0.07329098,
         2.49601294, 10.46037923],
       [ 0.23568602,  0.03850576,  0.24219178, ...,  0.02958904,
         2.77998462, 11.3921849 ],
       ...,
       [ 0.1284616 ,  0.07702359,  0.12747784, ...,  0.11619662,
         1.40488967,  5.84892713],
       [ 0.20124332,  0.04528579,  0.20904348, ...,  0.02713044,
         3.00014512, 12.82905908],
       [ 0.1843094 ,  0.04314487,  0.18698413, ...,  0.04095238,
         2.07517227,  6.90485956]])

In [28]:
logit = LogisticRegression(tol=0.00001)
logit.fit(x_train, y_train)
y_pred_logit = logit.predict(x_test)


C:\Users\User\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [29]:
accuracy_score(y_test, y_pred_logit)*100

88.64353312302839

In [30]:
forest = RandomForestClassifier(n_estimators=100, random_state=0)
forest.fit(x_train, y_train)
y_pred_r=forest.predict(x_test)

In [31]:
accuracy_score(y_test, y_pred_r)*100

94.00630914826499

In [14]:
from sklearn import svm

In [15]:
model=svm.SVC(kernel='linear')
model.fit(x_train,y_train)
y_pred_s=model.predict(x_test) 

In [35]:
accuracy_score(y_test, y_pred_s)*100

92.58675078864354

In [36]:
import xgboost
model = xgboost.XGBClassifier()
model.fit(x_train,y_train)
y_pred_x=model.predict(x_test)

In [37]:
accuracy_score(y_test, y_pred_x)*100

98.42271293375394

In [32]:
import sys
from aubio import source, pitch

win_s = 4096
hop_s = 512 

s = source("C:\\Users\\User\\Documents\\Gender_prediction\\male_v\\male1.wav", win_s, hop_s)
samplerate = s.samplerate

tolerance = 0.8

pitch_o = pitch("yin", win_s, hop_s, samplerate)
pitch_o.set_unit("midi")
pitch_o.set_tolerance(tolerance)

pitches = []
confidences = []

total_frames = 0
while True:
    samples, read = s()
    pitch = pitch_o(samples)[0]
    pitches += [pitch]
    confidence = pitch_o.get_confidence()
    confidences += [confidence]
    total_frames += read
    if read < hop_s: break

print("Average frequency = " + str(np.array(pitches).mean()) + " hz")



Average frequency = 59.667877 hz


In [33]:
import numpy as np

def spectral_properties(y: np.ndarray, fs: int) -> dict:
    spec = np.abs(np.fft.rfft(y))
    freq = np.fft.rfftfreq(len(y), d=1 / fs)
    spec = np.abs(spec)
    amp = spec / spec.sum()
    mean = np.mean(y)#(freq * amp).sum()
    sd = np.std(y)#np.sqrt(np.sum(amp * ((freq - mean) ** 2)))
    amp_cumsum = np.cumsum(amp)
    median = np.median(y)#freq[len(amp_cumsum[amp_cumsum <= 0.5]) + 1]
    mode = freq[amp.argmax()]
    Q25 = freq[len(amp_cumsum[amp_cumsum <= 0.25]) + 1]
    Q75 = freq[len(amp_cumsum[amp_cumsum <= 0.75]) + 1]
    #Q75,Q25=np.percentile(freq, [75 ,25])
    IQR = Q75 - Q25
    z = amp - amp.mean()
    w = amp.std()
    skew = ((z ** 3).sum() / (len(spec) - 1)) / w ** 3
    kurt = ((z ** 4).sum() / (len(spec) - 1)) / w ** 4

    result_d = {
        'mean': mean/1000,
        'sd': sd/1000,
        'median': median/1000,
        'mode': mode,
        'Q25': Q25/1000,
        'Q75': Q75/1000,
        'IQR': IQR/1000,
        'skew': skew,
        'kurt': kurt
    }

    return result_d

In [34]:
dits={}
dicts=spectral_properties(np.array(pitches), samplerate)

In [35]:
dicts

{'mean': 0.05966787719726562,
 'sd': 0.02132032012939453,
 'median': 0.06816697692871093,
 'mode': 0.0,
 'Q25': 0.024236686390532544,
 'Q75': 1.1633609467455621,
 'IQR': 1.1391242603550298,
 'skew': 8.924390642628518,
 'kurt': 81.13707973264559}

In [37]:
x_test_ll=[]
for i,x in dicts.items():
    x_test_ll.append(x)
x_test_ll=np.array(x_test_ll)
x_test_ll=x_test_ll.reshape(1, -1)
y_pred_r=forest.predict(x_test_ll)
if(y_pred_r==1):
    print("Male Voice")
else:
    print("Female Voice")


Male Voice


In [168]:
import csv
with open('mydata.csv','a',newline='') as file:
    writer=csv.writer(file)
    #writer.writerow(['meanfreq','sd','median','mode','Q25','Q75','IQR','skew','kurt','label'
#])
    #for i, x in enumerate(y_pred_l):
    writer.writerow([dicts['mean'],dicts['sd'],dicts['median'],dicts['mode'],dicts['Q25'],dicts['Q75'],dicts['IQR'],
                     dicts['skew'],dicts['kurt'],'female'])

In [162]:
dicts['mean']

0.02383847236633301